# About

Convert ssd_mobilenet_v2_coco_2018_03_29 to Core ML

Try optimizing the TF graph befor conversion to Core ML.
Failed with error:
```
ValueError: Input 0 of node ToFloat was passed float from image_tensor:0 incompatible with expected uint8.
```

# Download model and setup environment

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
!tar -zxvf ssd_mobilenet_v2_coco_2018_03_29.tar.gz

--2021-08-12 07:05:10--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.81.208, 2607:f8b0:4004:83f::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.81.208|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 187925923 (179M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’

ssd_mobilenet_v2_co 100%[===================>] 179.22M   195MB/s    in 0.9s    

2021-08-12 07:05:11 (195 MB/s) - ‘ssd_mobilenet_v2_coco_2018_03_29.tar.gz’ saved [187925923/187925923]

ssd_mobilenet_v2_coco_2018_03_29/checkpoint
ssd_mobilenet_v2_coco_2018_03_29/model.ckpt.meta
ssd_mobilenet_v2_coco_2018_03_29/pipeline.config
ssd_mobilenet_v2_coco_2018_03_29/saved_model/saved_model.pb
ssd_mobilenet_v2_coco_2018_03_29/frozen_inference_graph.pb
ssd_mobilenet_v2_coco_2018_03_29/saved_model/
ssd_mobilenet_v2_coco_2018_03_29/saved_mo

In [ ]:
!pip install coremltools==4.1

     |████████████████████████████████| 3.4 MB 34.1 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=9458d39d0f19cdd1fa2af944987eb1d44daf3a12142fbf9822b2450e519ae2a1
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [ ]:
%tensorflow_version 2.x

import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import coremltools as ct
print(ct.__version__)

3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
2.5.0


4.1


# Inspect the TF graph

In [ ]:
def load_frozen_graph(path):
  with tf.io.gfile.GFile(path, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

    return graph_def


def load_saved_model(path):
  the_graph = tf.Graph()
  with tf.compat.v1.Session(graph=the_graph) as sess:
    tags = [tf.compat.v1.saved_model.tag_constants.SERVING]
    tf.compat.v1.saved_model.loader.load(sess, tags, path)
  return the_graph


def export_ops_name(the_graph, filename):
  with open(filename, "w") as text_file:
    ops = the_graph.get_operations()
    N = len(ops)
    for i in range(N):
      text_file.write('\n\nop id {} , op type: "{}"'.format(str(i), ops[i].type))
      
      text_file.write('\ninput(s):'),
      for x in ops[i].inputs:
        text_file.write("name = {}, shape: {}, ".format(x.name, x.get_shape()))
      
      text_file.write('\noutput(s):'),
      for x in ops[i].outputs:
        text_file.write("name = {}, shape: {},".format(x.name, x.get_shape()))

  print('Exported to', filename)


model_name = 'ssd_mobilenet_v2_coco_2018_03_29'
saved_model_dir = '/content/ssd_mobilenet_v2_coco_2018_03_29/saved_model'

the_graph = load_saved_model(saved_model_dir)
export_ops_name(the_graph, f'{model_name}_ops.txt')



Exported to ssd_mobilenet_v2_coco_2018_03_29_ops.txt


# Optimize TF graph

In [ ]:
!rm -rf optimized_saved_model

from tensorflow.python.tools import strip_unused_lib
from tensorflow.python.framework import dtypes
from tensorflow.python.platform import gfile

from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants


input_node = "image_tensor"
boxes_output_node = "detection_boxes"
scores_output_node = "detection_scores"
optimized_saved_model_dir = 'optimized_saved_model'


def print_node_name(graph_def):
  nodes = [n.name for n in graph_def.node]
  print('nodes', len(nodes), nodes)
  print(nodes[0], '->', nodes[-1])


def optimize_graph(graph_def, 
                   input_node_names, 
                   output_node_names,
                   export_dir):

  print('nodes before strip_unused_lib')
  print_node_name(graph_def)
  
  graph_def = strip_unused_lib.strip_unused(
          input_graph_def = graph_def,
          input_node_names = input_node_names,
          output_node_names = output_node_names,
          placeholder_type_enum = dtypes.float32.as_datatype_enum)
  
  print('nodes after strip_unused_lib')
  print_node_name(graph_def)

  builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_dir)
  sigs = {}
  
  with tf.compat.v1.Session(graph=tf.Graph()) as sess:
      tf.import_graph_def(graph_def, name="")
      g = tf.compat.v1.get_default_graph()
      inputs = {n:g.get_tensor_by_name(n+':0') for n in input_node_names}
      outputs = {n:g.get_tensor_by_name(n+':0') for n in output_node_names}

      sigs[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = \
        tf.compat.v1.saved_model.signature_def_utils.predict_signature_def(inputs, outputs)

      builder.add_meta_graph_and_variables(sess, [tag_constants.SERVING], signature_def_map=sigs)
      builder.save()


input_nodes=[input_node]
output_nodes=[boxes_output_node, scores_output_node]
optimize_graph(the_graph.as_graph_def(), input_nodes, output_nodes, optimized_saved_model_dir)


nodes before strip_unused_lib
nodes 7975 ['Const', 'image_tensor', 'ToFloat', 'Preprocessor/map/Shape', 'Preprocessor/map/strided_slice/stack', 'Preprocessor/map/strided_slice/stack_1', 'Preprocessor/map/strided_slice/stack_2', 'Preprocessor/map/strided_slice', 'Preprocessor/map/TensorArray', 'Preprocessor/map/TensorArrayUnstack/Shape', 'Preprocessor/map/TensorArrayUnstack/strided_slice/stack', 'Preprocessor/map/TensorArrayUnstack/strided_slice/stack_1', 'Preprocessor/map/TensorArrayUnstack/strided_slice/stack_2', 'Preprocessor/map/TensorArrayUnstack/strided_slice', 'Preprocessor/map/TensorArrayUnstack/range/start', 'Preprocessor/map/TensorArrayUnstack/range/delta', 'Preprocessor/map/TensorArrayUnstack/range', 'Preprocessor/map/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3', 'Preprocessor/map/Const', 'Preprocessor/map/TensorArray_1', 'Preprocessor/map/TensorArray_2', 'Preprocessor/map/while/Enter', 'Preprocessor/map/while/Enter_1', 'Preprocessor/map/while/Enter_2', 'Prepro

ValueError: ignored